In [53]:
from util import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,\
    Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from keras.optimizers import Adam
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding


 
# Define the corpus
# corpus = ['The cat sat on the mat',
#           'The dog ran in the park',
#           'The bird sang in the tree']
train_corpus = open("dataset/train_clean.txt", "r",encoding='utf8').read().split(".")
train_noDiactric, train_labels =extract_data(train_corpus)
# Convert the corpus to a sequence of integers
tokenizer = Tokenizer(oov_token=1)
tokenizer.fit_on_texts(train_noDiactric)
train_sequences = tokenizer.texts_to_sequences(train_noDiactric)
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
word2idx['UKN']= len(word2idx) +1

In [2]:
# freq=tokenizer.word_counts
# least_10_words = sorted(freq.items(), key=lambda x: x[1])[:10]
# # print(least_10_words)
# for word, _ in least_10_words:
#     # print(word, freq)
#     del word2idx[word]

In [3]:

# val_corpus = open("dataset/val_clean.txt", "r",encoding='utf8').read().split(".")
# val_noDiactric, val_labels =extract_data(val_corpus)
# # Convert the corpus to a sequence of integers
# val_sequences = tokenizer.texts_to_sequences(val_noDiactric)
# # val_sequences= []
# # for i,sentence in enumerate(val_noDiactric):
# #     val_sequences.append(list())
# #     for word in sentence:
# #         if word not in word2idx:
# #             val_sequences[i].append(word2idx['UKN'])
# #         else: 
# #             val_sequences[i].append(word2idx[word])

In [4]:
# print(word2idx['قوله'])
# print(val_noDiactric[0])
# print(train_noDiactric[0])
# print(val_sequences[0])
# print(train_sequences[0])
# print(len(train_sequences))
# print(len(train_corpus))

In [5]:
# Define the parameters
# I think change to +2?
# vocab_size = len(tokenizer.word_index) + 1
vocab_size = len(tokenizer.word_index) + 2
embedding_size = 128 # was originally 10
window_size = 4
print(vocab_size)
learning_rate = 0.002
clipnorm = 1.0
 
# Generate the context-target pairs
contexts = []
targets = []
print(len(train_sequences))
for sequence in train_sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i - window_size:i] +\
            sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)
val_conexts = []
val_targets = []
for sequence in val_sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i - window_size:i] +\
            sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        val_conexts.append(context)
        val_targets.append(target)
# Convert the contexts and targets to numpy arrays
X_train = np.array(contexts)
Y_train = np.array(targets)
X_val = np.array(val_conexts)
Y_val = np.array(val_targets)
print(X_train.shape)
print(Y_train.shape)

 
# Define the CBOW model
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(units=vocab_size, activation='softmax'))

adam_optimizer = Adam(learning_rate=learning_rate, clipnorm=clipnorm)
model.compile(loss='sparse_categorical_crossentropy',optimizer=adam_optimizer)
model.summary()


In [6]:
# history = model.fit(X_train, Y_train, epochs=30, batch_size=3500, validation_data=(X_val, Y_val))

In [7]:
# # model.save_weights('cbow_weights.h5')
# # model.save('cbow.h5') 
 
# # Load the pre-trained weights
# # model.load_weights('cbow_weights.h5')
# model = load_model('cbow.h5')

In [8]:

# # Get the word embeddings
# embeddings = model.get_weights()[0]
 
# # Perform PCA to reduce the dimensionality
# # of the embeddings
# pca = PCA(n_components=2)
# reduced_embeddings = pca.fit_transform(embeddings)
 
# # Visualize the embeddings
# plt.figure(figsize=(5, 5))
# for i, word in enumerate(tokenizer.word_index.keys()):
#     x, y = reduced_embeddings[i]
#     plt.scatter(x, y)
#     plt.annotate(word, xy=(x, y), xytext=(5, 2),
#                  textcoords='offset points',
#                  ha='right', va='bottom')
# plt.show()

# Skip Gram

In [9]:
# # Removed due to bad results
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Embedding, Dense, Reshape
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import skipgrams


# train_corpus = open("dataset/train_clean.txt", "r",encoding='utf8').read()
# train_noDiactric = remove_diacritics(train_corpus).split(".")

# # word2idx = tokenizer.word_index
# # idx2word = {v:k for k, v in word2idx.items()}
# # word2idx['UKN']= len(word2idx) +1




# # Tokenize the text
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(train_noDiactric)
# total_words = len(tokenizer.word_index) + 1

# # Create input sequences and labels using skipgrams
# train_sequences = tokenizer.texts_to_sequences(train_noDiactric)
# skip_grams = [skipgrams(sequence, vocabulary_size=total_words, window_size=3) for sequence in train_sequences]

# data = []

# for (target, context), label in zip(skip_grams[0][0], skip_grams[0][1]):
#     data.append([target, context, label])

# target_word = Input((1,), dtype='int32')
# context_word = Input((1,), dtype='int32')

# # Define the embedding layer
# embedding_size = 50
# embedding_layer = Embedding(input_dim=total_words, output_dim=embedding_size, input_length=1)

# # Connect the target and context words to the embedding layer
# target_embed = embedding_layer(target_word)
# context_embed = embedding_layer(context_word)

# # Reshape the embeddings
# target_embed = Reshape((embedding_size, 1))(target_embed)
# context_embed = Reshape((embedding_size, 1))(context_embed)

# # Dot product between target and context embeddings
# dot_product = tf.keras.layers.Dot(axes=1)([target_embed, context_embed])
# dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)

# # Output layer
# output = Dense(1, activation='sigmoid')(dot_product)

# # Build the Skip-Gram model
# model = Model(inputs=[target_word, context_word], outputs=output)
# model.compile(loss='binary_crossentropy', optimizer='adam')

# # Convert data to numpy arrays
# target_data = np.array([pair[0] for pair in data])
# context_data = np.array([pair[1] for pair in data])
# labels = np.array([pair[2] for pair in data])
# print(target_data.shape)
# print(context_data.shape)
# print(labels.shape)

# # Train the model using model.fit
# model.fit([target_data, context_data], labels, epochs=10, batch_size=32)

# # Get the word embeddings
# word_embeddings = embedding_layer.get_weights()[0]


In [17]:

max_len = 500
EXTRACT_FEATRURES = True 

train_dir = 'Dataset/train_clean.txt'
val_dir = 'Dataset/val_clean.txt'

with open(train_dir, 'r', encoding='utf8') as f:
    train = f.read()
with open(val_dir, 'r', encoding='utf8') as f:
    val = f.read()


X_train_dir = 'pkl_dir/X_train.txt'
X_val_dir = 'pkl_dir/X_val.txt'
y_train_dir = 'pkl_dir/y_train.txt'
y_val_dir = 'pkl_dir/y_val.txt'

if EXTRACT_FEATRURES:
    X_train, y_train = extract_features(train, max_len)
    X_val, y_val = extract_features(val, max_len)
else:
    with open(X_train_dir, 'rb') as f:
        X_train = pickle.load(f)
    with open(y_train_dir, 'rb') as f:
        y_train = pickle.load(f)
    with open(X_val_dir, 'rb') as f:
        X_val = pickle.load(f)
    with open(y_val_dir, 'rb') as f:
        y_val = pickle.load(f)




sentence_encoder = LabelEncoder().fit(X_train.flatten())
X_train = sentence_encoder.transform(X_train.flatten()).reshape(X_train.shape).astype(np.uint8)
X_val = sentence_encoder.transform(X_val.flatten()).reshape(X_val.shape).astype(np.uint8)





In [79]:
# latent_dim = 256
# num_encoder_tokens = num_decoder_tokens = np.unique(X_train).shape[0]
# vocab_size = num_encoder_tokens  # Assuming vocab size is the number of unique tokens
# embed_dim = 25  # Dimension of embedding

# from keras.layers import Dense, Activation
# from keras.layers import concatenate, dot
# from keras.layers import Add, Dot, RepeatVector, Activation, Lambda
# from keras.layers import Permute, Multiply, Reshape, Concatenate


# from keras.layers import Attention, BatchNormalization

# # Define an input sequence and process it.
# input1 = Input(shape=(None,))
# input2 = Input(shape=(None,))

# # Add embedding layer
# embedded1 = Embedding(vocab_size, embed_dim)(input1)
# embedded2 = Embedding(vocab_size, embed_dim)(input2)

# encoder = LSTM(latent_dim, return_sequences=True, return_state=True)
# encoder_outputs, state_h, state_c = encoder(embedded1)

# # Add batch normalization to the encoder
# encoder_outputs = BatchNormalization()(encoder_outputs)

# encoder_states = [state_h, state_c]

# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, num_decoder_tokens))
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(embedded2, initial_state=encoder_states)

# # Add attention
# attention = Attention()([decoder_outputs, encoder_outputs])

# # Concatenate the context vector with the decoder output
# decoder_concat_input = concatenate([attention, decoder_outputs])

# # Add batch normalization to the decoder
# decoder_concat_input = BatchNormalization()(decoder_concat_input)

# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_concat_input)

# # Define the model
# model = Model([input1, input2], decoder_outputs)

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
# print(X_train.shape)
SOS_encode = sentence_encoder.transform(np.array([ord(SOS)]).flatten())[0]
X_train_r = np.concatenate((np.full((X_train.shape[0], 1), SOS_encode), X_train[:, :-1]), axis=1)
# X_train_r = np.expand_dims(X_train_r, axis=2)
# X_train = np.expand_dims(X_train, axis=2)
print(X_train_r.shape)

# Do it for validation data as well
X_val_r = np.concatenate((np.full((X_val.shape[0], 1), SOS_encode), X_val[:, :-1]), axis=1)
# X_val_r = np.expand_dims(X_val_r, axis=2)
# X_val = np.expand_dims(X_val, axis=2)
print(X_val_r.shape)

(39959, 500)
(1991, 500)


In [83]:
model.fit([X_train, X_train_r], X_train, epochs=50, batch_size=200, validation_data=([X_val, X_val_r], X_val))

Epoch 1/50
200/200 [==============================] - 26s 124ms/step - loss: 1.0115 - accuracy: 0.7264 - val_loss: 1.0025 - val_accuracy: 0.7287
Epoch 2/50
200/200 [==============================] - 24s 122ms/step - loss: 0.9915 - accuracy: 0.7307 - val_loss: 0.9861 - val_accuracy: 0.7316
Epoch 3/50
200/200 [==============================] - 25s 124ms/step - loss: 0.9743 - accuracy: 0.7334 - val_loss: 0.9646 - val_accuracy: 0.7354
Epoch 4/50
200/200 [==============================] - 26s 130ms/step - loss: 0.9564 - accuracy: 0.7366 - val_loss: 0.9479 - val_accuracy: 0.7387
Epoch 5/50
200/200 [==============================] - 25s 125ms/step - loss: 0.9405 - accuracy: 0.7398 - val_loss: 0.9315 - val_accuracy: 0.7415
Epoch 6/50
200/200 [==============================] - 25s 126ms/step - loss: 0.9245 - accuracy: 0.7432 - val_loss: 0.9169 - val_accuracy: 0.7456
Epoch 7/50
200/200 [==============================] - 25s 126ms/step - loss: 0.9131 - accuracy: 0.7459 - val_loss: 0.9037 - val_ac

KeyboardInterrupt: 

In [87]:
# model.save('embeddings.h5') 

 
encoder_model = load_model('embeddings.h5')
# print(model.summary())
# encoder_model = Model(input1, encoder_states)
# encoder_model.summary()
# encoder_model.save('encoder.h5')


In [86]:
modelcbow = load_model('cbow.h5')
print(modelcbow.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 8, 128)            13774976  
                                                                 
 lambda_4 (Lambda)           (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 107617)            13882593  
                                                                 
Total params: 27657569 (105.51 MB)
Trainable params: 27657569 (105.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [90]:
from keras.models import Model
from keras.layers import Input, LSTM, TimeDistributed, Dense, Embedding, Concatenate, BatchNormalization, Dropout

embedding_size = 128
window_size = 4
latent_dim = 256
num_encoder_tokens = num_decoder_tokens = np.unique(X_train).shape[0]
vocab_size = len(tokenizer.word_index) + 2
print(vocab_size)
embed_dim = 25

input_tensor = Input(shape=(None,))

# Define the LSTM part
embedding_lstm = Embedding(num_encoder_tokens, embed_dim)(input_tensor)
lstm_output = LSTM(latent_dim, return_sequences=True)(embedding_lstm)

# Define the CBOW part
embedding_cbow = Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=2*window_size)(input_tensor)

# Concatenate the outputs
concatenated = Concatenate()([lstm_output, embedding_cbow])

# Add another LSTM
lstm_1 = LSTM(256, return_sequences=True)(concatenated)
bn_1 = BatchNormalization()(lstm_1)
dropout_1 = Dropout(rate=0.5)(bn_1)


dense_1 = TimeDistributed(Dense(256, activation='relu'))(dropout_1)
bn_3 = BatchNormalization()(dense_1)

dense_2 = TimeDistributed(Dense(256, activation='relu'))(bn_3)
bn_4 = BatchNormalization()(dense_2)

output = TimeDistributed(Dense(np.unique(y_train).shape[0], activation='softmax'))(bn_4)

# Define the final model
final_model = Model(inputs=input_tensor, outputs=output)

# print(encoder_model.summary())
print(final_model.summary())

# Set weights for first 2 layers from the trained model
final_model.layers[1].set_weights(encoder_model.layers[2].get_weights())  # Embedding LSTM
final_model.layers[2].set_weights(encoder_model.layers[4].get_weights())  # LSTM
final_model.layers[3].set_weights(modelcbow.layers[0].get_weights())  # cbow Embedding 

final_model.layers[1].trainable = False
final_model.layers[2].trainable = False
final_model.layers[3].trainable = False



final_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


107617
Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_35 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_58 (Embedding)    (None, None, 25)             1400      ['input_35[0][0]']            
                                                                                                  
 lstm_77 (LSTM)              (None, None, 256)            288768    ['embedding_58[0][0]']        
                                                                                                  
 embedding_59 (Embedding)    (None, None, 128)            1377497   ['input_35[0][0]']            
                                                          6                         

In [91]:
final_model.fit(X_train, y_train, epochs=50, batch_size=200, validation_data=(X_val, y_val))

Epoch 1/50
200/200 [==============================] - 39s 182ms/step - loss: 0.4705 - accuracy: 0.8605 - val_loss: 0.4983 - val_accuracy: 0.8377
Epoch 2/50
200/200 [==============================] - 36s 181ms/step - loss: 0.3280 - accuracy: 0.8964 - val_loss: 0.3206 - val_accuracy: 0.9011
Epoch 3/50
200/200 [==============================] - 36s 182ms/step - loss: 0.2949 - accuracy: 0.9077 - val_loss: 0.2740 - val_accuracy: 0.9155
Epoch 4/50
200/200 [==============================] - 37s 183ms/step - loss: 0.2775 - accuracy: 0.9137 - val_loss: 0.2609 - val_accuracy: 0.9195
Epoch 5/50
200/200 [==============================] - 37s 184ms/step - loss: 0.2345 - accuracy: 0.9186 - val_loss: 0.2030 - val_accuracy: 0.9255
Epoch 6/50
200/200 [==============================] - 37s 184ms/step - loss: 0.2105 - accuracy: 0.9229 - val_loss: 0.1964 - val_accuracy: 0.9279
Epoch 7/50
200/200 [==============================] - 37s 187ms/step - loss: 0.2043 - accuracy: 0.9252 - val_loss: 0.1909 - val_ac

KeyboardInterrupt: 

In [92]:
final_model.save('manualFeatures.h5') 

 
# model_manualFeatures = load_model('manualFeatures.h5')



/home/ali/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
